### Информация
- **Автор**: Зитцер Данил
- **Дата создания**: 5 мая 2021 года, 09:58:13
- **Краткое описание**: С помощью cv2 напишем распознавальщик геометрических фигур.
- **Полезные ссылки**:
    - [Contours shape recognition](https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/blob/master/Chapter08/01-chapter-content/contours_shape_recognition.py)
    - [Contours matching](https://github.com/PacktPublishing/Mastering-OpenCV-4-with-Python/blob/master/Chapter08/01-chapter-content/contours_matching.py)
    - []()
- **Выводы**:

In [9]:
import cv2
import numpy as np

from nptyping import NDArray
from pydantic import BaseModel, PositiveInt, validator
from typing import Any, Dict, List, Tuple, Union 

from IPython.display import clear_output

### Константы 

In [2]:
IMAGE_H = 500
IMAGE_W = 500

### Shape generator

In [3]:
class Painter(BaseModel):
    image_h: PositiveInt
    image_w: PositiveInt
        
    @property
    def imsize(self) -> NDArray[(2, ), np.uint32]:
        return np.array([self.image_h, self.image_w], dtype=np.uint32)
    
    @property
    def imshape(self) -> NDArray[(3, ), np.uint32]:
        return np.array([self.image_h, self.image_w, 3], dtype=np.uint32)
    
    @property
    def random_color(self) -> NDArray[(3, ), np.uint8]:
        return np.random.randint(low=0, high=255, size=3, dtype=np.uint8).tolist()
    
    @property
    def black_background(self) -> NDArray[(Any, Any, 3), np.uint8]:
        return np.zeros(shape=self.imshape, dtype=np.uint8)
    
    def draw_random_square(self) -> NDArray[(Any, Any, 3), np.uint8]:
        p1 = np.random.randint(low=0, high=self.imsize.min(), size=2)
        p2 = p1 + np.random.randint(low=0, high=(self.imsize - p1).min())

        return cv2.rectangle(self.black_background, tuple(p1), tuple(p2), self.random_color, -1)

    def draw_random_circle(self) -> NDArray[(Any, Any, 3), np.uint8]:
        radius = np.random.randint(low=0, high=self.imsize.min() // 2 - 1)
        center = np.random.randint(low=0+radius, high=self.imsize.min()-radius, size=2)

        return cv2.circle(self.black_background, tuple(center), radius, self.random_color, -1)
    
    def draw_random_shape(self) -> NDArray[(Any, Any, 3), np.uint8]:
        return self.draw_random_circle() if np.random.rand() > 0.5 else self.draw_random_square()

In [4]:
painter = Painter(image_h=IMAGE_H, image_w=IMAGE_W)

In [5]:
for i in range(5):
    cv2.imshow('image', painter.draw_random_shape())
    cv2.waitKey(3000)    
cv2.destroyAllWindows()

### Shape recognition

In [6]:
def shape_recognizer(image: NDArray[(Any, Any, 3), np.uint8]) -> str:
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image_thresh = cv2.threshold(image_gray, 10, 255, cv2.THRESH_BINARY)
    image_contours, _ = cv2.findContours(image_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if len(image_contours) > 0:
        contour = image_contours[0]
    else:
        return 'shape not found'
    
    perimeter = cv2.arcLength(contour, True)
    contour_approx = cv2.approxPolyDP(contour, 0.03*perimeter, True)
    num_vertices = len(contour_approx)

    return {
        num_vertices < 3: 'unknown',
        num_vertices == 3: 'triangle',
        num_vertices == 4: 'rectangle',
        num_vertices == 5: 'pentagon',
        num_vertices == 6: 'hexagon',
        num_vertices > 6: 'circle',
    }[True]

In [7]:
painter = Painter(image_h=IMAGE_H, image_w=IMAGE_W)

In [8]:
image = painter.draw_random_shape()

while True:
    image = painter.draw_random_shape()
    print(shape_recognizer(image))
    
    cv2.imshow('image', image)
    
    if cv2.waitKey(3000) & 0xFF == ord('q'):
        break
        
    clear_output(True)
    
cv2.destroyAllWindows()

rectangle
